In [1]:
from sqlalchemy import create_engine, MetaData, select
import datetime as dt
import pandas as pd
import mplfinance as mpf


PG_USER = "postgres"
PG_PASSWORD = "data123"
PG_HOSTNAME = "127.0.0.1"
PG_PORT = "5432"
PG_DB_NAME = "dailypricedata"

connect = f"postgresql+psycopg2://{PG_USER}:{PG_PASSWORD}@{PG_HOSTNAME}:{PG_PORT}/{PG_DB_NAME}"
engine = create_engine(connect)

meta_data = MetaData(bind=engine)
MetaData.reflect(meta_data)
nsedata = meta_data.tables['NSEData']
query = select([
    nsedata.c.Symbol
])

stockNameList = engine.execute(query).fetchall()


In [2]:
def getStockData(engine, script):
    query = f"select * from \"{script}\""
    df = pd.read_sql_query(query, engine)
    df = pd.DataFrame(df)
    #df = df.set_index('Date')
    df["prevClose"] = df.Close.shift(1)
    df['change'] = df[['Open', 'prevClose']].pct_change(axis=1)['prevClose']
    
    return df

In [29]:
import datetime
import pendulum
startDate = pendulum.now().add(days=-30).format('YYYY-MM-DD')
granddf = pd.DataFrame()
percentageGap = 5
for script in stockNameList:
    resultdf = getStockData(engine, script[0])
    df = resultdf
    df['Date'] = pd.to_datetime(df['Date'])
    df = df[df.Date > startDate]
    df = df[df.change < -percentageGap/100]
    df['Stock'] = script[0]
    granddf = pd.concat([granddf, df])
df = granddf


/tmp/ipykernel_30139/3520069541.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negativeGap['Stock'] = script[0]
/tmp/ipykernel_30139/3520069541.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negativeGap['Stock'] = script[0]
/tmp/ipykernel_30139/3520069541.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

In [30]:
granddfpositiveGap.sort_values(by=['Date'])
granddfnegativeGap.sort_values(by=['Date'])


,Date,Open,High,Low,Close,Adj Close,Volume,prevClose,change,Stock
441,2022-07-18,357.000000,363.799988,347.649994,352.700012,352.700012,2264162.0,380.600006,0.066106,NIITLTD
442,2022-07-19,20.049999,20.049999,20.049999,20.049999,20.049999,2173945.0,21.100000,0.052369,ALOKINDS
443,2022-07-20,280.000000,283.000000,276.700012,278.850006,278.850006,3156045.0,298.649994,0.066607,HINDZINC
444,2022-07-21,2290.000000,2373.800049,2180.000000,2329.699951,2329.699951,4500329.0,2473.350098,0.080066,GLAND
445,2022-07-22,1051.050049,1054.900024,1026.050049,1031.250000,951.250000,110312.0,1143.050049,0.087532,GOODYEAR
446,2022-07-25,64.699997,64.699997,62.400002,63.200001,63.200001,11178314.0,68.400002,0.057187,HFCL
446,2022-07-25,316.500000,323.700012,312.399994,315.100006,315.100006,856142.0,338.649994,0.069984,TINPLATE
447,2022-07-26,240.000000,244.899994,238.850006,241.399994,241.399994,18479077.0,258.000000,0.075000,VEDL
306,2022-07-26,2222.199951,2275.000000,2160.000000,2258.300049,2258.300049,133469.0,2370.500000,0.066736,TATVA
447,2022-07-26,5.750000,5.750000,5.750000,5.750000,5.750000,873381.0,6.050000,0.052174,FRETAIL
